FIRST DISCUSSION LEAD US TO...

### 1) EMBEDDING NN (first in time)

- create embeddings for each sentence of "context": bert (?)
- represent "context" by these embeddings

OPEN QUESTION: embedding of the sentence by sentence or by words?
______

### 2) ANSWERING NN (second in time)

a) FIRST APPROACH : OBSERVING GENERAL INFORMATION
  - NN
    - input : (whole "context", question)
    - output : index of answer(readapted)? direct answer?

b) SECOND APPROACH : FOCUSING ON SPECIFIC INFORMATION
  - confront each sentence embedding to embedded question: cosine similarity?
    - to find which sentence contains the answer to the question
  - NN
    - input : (sentence of "context", question)
    - output : index of answer(readapted)? direct answer?



______
______

NB : FINAL ANSWERS HAVE TO BE [index, text], THEREFORE WE NEED TO FIND THEM BOTH IN ANY CASE...


In [ ]:
# find out "answer_start" meaning :

t = """Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms	upraised with the legend \"Venite Ad Me Omnes\". Next to the Main Building is the Basilica of the	Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858."""
print(t[515:])

Saint Bernadette Soubirous in 1858.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# system packages
import os
import shutil
import sys

# data and numerical management packages
import pandas as pd
import numpy as np

# useful during debugging (progress bars)
from tqdm import tqdm


from urllib import request
import tarfile

In [ ]:
# #Loading the data 
# import json 
# import os


# # DATAFRAME ROWS
# dataframerows = []

# with open('/content/drive/My Drive/NLP/final_project/training_set.json') as f:
#     data = json.load(f)

#     for el in data['data']:
#         title = el['title']
#         paragraphs = el['paragraphs']
        
#         for context_qas in paragraphs:
#               context = context_qas['context'] 
#               qas = context_qas['qas']

#               for a_q in qas: #[0]["answers"][0]["text"]
#                   answer = a_q['answers']
#                   question = a_q['question']
#                   id = a_q['id']
                  
#                   for a in answer:
#                       text = a['text']
#                       answer_start = a['answer_start']
                      
#                       # CREATE ROW
#                       row = {
#                           "title" : title,
#                           #"paragraphs" : paragraphs,
#                           #"context_qas" : context_qas,
#                           #"context" : context,
#                           #"qas" : qas,
#                           "id" : id,
#                           #"question" : question,
#                           #"answer" : answer,
#                           #"text" : text,
#                           #"answer_start" : answer_start,
#                           }
                      
#                       # APPEND ROW TO DATAFRAME ROWS
#                       dataframerows.append(row)

#     df = pd.DataFrame(dataframerows)
#     df = df[["title", "id"]]

In [ ]:
#Loading the data 
import json 
import os


# DATAFRAME ROWS
dataframerows = []

with open('/content/drive/My Drive/NLP/final_project/training_set.json') as f:
    data = json.load(f)

    for el in data['data']:
        title = el['title']
        paragraphs = el['paragraphs']
        
        for context_qas in paragraphs:
              context = context_qas['context'] 
              qas = context_qas['qas']

              for a_q in qas: #[0]["answers"][0]["text"]
                  answer = a_q['answers']
                  question = a_q['question']
                  id = a_q['id']
                  
                  for a in answer:
                      text = a['text']
                      answer_start = a['answer_start']
                      
                      # CREATE ROW
                      row = {
                          "title" : title,
                          "paragraphs" : paragraphs,
                          "context_qas" : context_qas,
                          "context" : context,
                          "qas" : qas,
                          "id" : id,
                          "question" : question,
                          "answer" : answer,
                          "text" : text,
                          "answer_start" : answer_start,
                          }
                      
                      # APPEND ROW TO DATAFRAME ROWS
                      dataframerows.append(row)

    df = pd.DataFrame(dataframerows)
    df = df[["title", "paragraphs", "context_qas", "context", "qas", "id", "question", "answer", "text", "answer_start"]]

In [ ]:
# A QUICK LOOK

print(df.columns)
print()
df.describe

Index(['title', 'paragraphs', 'context_qas', 'context', 'qas', 'id',
       'question', 'answer', 'text', 'answer_start'],
      dtype='object')



<bound method NDFrame.describe of                           title  ... answer_start
0      University_of_Notre_Dame  ...          515
1      University_of_Notre_Dame  ...          188
2      University_of_Notre_Dame  ...          279
3      University_of_Notre_Dame  ...          381
4      University_of_Notre_Dame  ...           92
...                         ...  ...          ...
87594                 Kathmandu  ...          229
87595                 Kathmandu  ...          414
87596                 Kathmandu  ...          476
87597                 Kathmandu  ...          199
87598                 Kathmandu  ...            0

[87599 rows x 10 columns]>

In [ ]:
df.head(10)

,title,paragraphs,context_qas,context,qas,id,question,answer,text,answer_start
0,University_of_Notre_Dame,"[{'context': 'Architecturally, the school has ...","{'context': 'Architecturally, the school has a...","Architecturally, the school has a Catholic cha...","[{'answers': [{'answer_start': 515, 'text': 'S...",5733be284776f41900661182,To whom did the Virgin Mary allegedly appear i...,"[{'answer_start': 515, 'text': 'Saint Bernadet...",Saint Bernadette Soubirous,515
1,University_of_Notre_Dame,"[{'context': 'Architecturally, the school has ...","{'context': 'Architecturally, the school has a...","Architecturally, the school has a Catholic cha...","[{'answers': [{'answer_start': 515, 'text': 'S...",5733be284776f4190066117f,What is in front of the Notre Dame Main Building?,"[{'answer_start': 188, 'text': 'a copper statu...",a copper statue of Christ,188
2,University_of_Notre_Dame,"[{'context': 'Architecturally, the school has ...","{'context': 'Architecturally, the school has a...","Architecturally, the school has a Catholic cha...","[{'answers': [{'answer_start': 515, 'text': 'S...",5733be284776f41900661180,The Basilica of the Sacred heart at Notre Dame...,"[{'answer_start': 279, 'text': 'the Main Build...",the Main Building,279
3,University_of_Notre_Dame,"[{'context': 'Architecturally, the school has ...","{'context': 'Architecturally, the school has a...","Architecturally, the school has a Catholic cha...","[{'answers': [{'answer_start': 515, 'text': 'S...",5733be284776f41900661181,What is the Grotto at Notre Dame?,"[{'answer_start': 381, 'text': 'a Marian place...",a Marian place of prayer and reflection,381
4,University_of_Notre_Dame,"[{'context': 'Architecturally, the school has ...","{'context': 'Architecturally, the school has a...","Architecturally, the school has a Catholic cha...","[{'answers': [{'answer_start': 515, 'text': 'S...",5733be284776f4190066117e,What sits on top of the Main Building at Notre...,"[{'answer_start': 92, 'text': 'a golden statue...",a golden statue of the Virgin Mary,92
5,University_of_Notre_Dame,"[{'context': 'Architecturally, the school has ...","{'context': 'As at most other universities, No...","As at most other universities, Notre Dame's st...","[{'answers': [{'answer_start': 248, 'text': 'S...",5733bf84d058e614000b61be,When did the Scholastic Magazine of Notre dame...,"[{'answer_start': 248, 'text': 'September 1876'}]",September 1876,248
6,University_of_Notre_Dame,"[{'context': 'Architecturally, the school has ...","{'context': 'As at most other universities, No...","As at most other universities, Notre Dame's st...","[{'answers': [{'answer_start': 248, 'text': 'S...",5733bf84d058e614000b61bf,How often is Notre Dame's the Juggler published?,"[{'answer_start': 441, 'text': 'twice'}]",twice,441
7,University_of_Notre_Dame,"[{'context': 'Architecturally, the school has ...","{'context': 'As at most other universities, No...","As at most other universities, Notre Dame's st...","[{'answers': [{'answer_start': 248, 'text': 'S...",5733bf84d058e614000b61c0,What is the daily student paper at Notre Dame ...,"[{'answer_start': 598, 'text': 'The Observer'}]",The Observer,598
8,University_of_Notre_Dame,"[{'context': 'Architecturally, the school has ...","{'context': 'As at most other universities, No...","As at most other universities, Notre Dame's st...","[{'answers': [{'answer_start': 248, 'text': 'S...",5733bf84d058e614000b61bd,How many student news papers are found at Notr...,"[{'answer_start': 126, 'text': 'three'}]",three,126
9,University_of_Notre_Dame,"[{'context': 'Architecturally, the school has ...","{'context': 'As at most other universities, No...","As at most other universities, Notre Dame's st...","[{'answers': [{'answer_start': 248, 'text': 'S...",5733bf84d058e614000b61c1,In what year did the student paper Common Sens...,"[{'answer_start': 908, 'text': '1987'}]",1987,908


In [ ]:
df.to_pickle('title_ids_complete.pkl')

In [ ]:
# A QUICK LOOK

print(df.columns)
print()
df.describe

Index(['title', 'paragraphs', 'context_qas', 'context', 'qas', 'id',
       'question', 'answer', 'text', 'answer_start'],
      dtype='object')



<bound method NDFrame.describe of                           title  ... answer_start
0      University_of_Notre_Dame  ...          515
1      University_of_Notre_Dame  ...          188
2      University_of_Notre_Dame  ...          279
3      University_of_Notre_Dame  ...          381
4      University_of_Notre_Dame  ...           92
...                         ...  ...          ...
87594                 Kathmandu  ...          229
87595                 Kathmandu  ...          414
87596                 Kathmandu  ...          476
87597                 Kathmandu  ...          199
87598                 Kathmandu  ...            0

[87599 rows x 10 columns]>

In [ ]:
'''# A QUICK LOOK TO THE DATA STRUCTURE

for i in range(3):
    print("Row", i, ":")
    for col in ["title", "context", "qas"]:
        print(col, ":", type(df[col][i]))
        print("\t", df[col][i])
        print()
    print()'''

'# A QUICK LOOK TO THE DATA STRUCTURE\n\nfor i in range(3):\n    print("Row", i, ":")\n    for col in ["title", "context", "qas"]:\n        print(col, ":", type(df[col][i]))\n        print("\t", df[col][i])\n        print()\n    print()'

In [ ]:
df_utils = df[['title', 'context', 'question', 'text', 'answer_start']]
df_utils.groupby('title').count()

,context,question,text,answer_start
title,,,,
2008_Sichuan_earthquake,521,521,521,521
2008_Summer_Olympics_torch_relay,500,500,500,500
51st_state,135,135,135,135
ASCII,110,110,110,110
A_cappella,104,104,104,104
...,...,...,...,...
Xbox_360,127,127,127,127
Yale_University,222,222,222,222
YouTube,162,162,162,162


In [ ]:
df_utils.loc[df['title']=="ASCII"]

,title,context,question,text,answer_start
30183,ASCII,"Originally based on the English alphabet, ASCI...",What is the ASCII based on?,English alphabet,24
30184,ASCII,"Originally based on the English alphabet, ASCI...",How many specific characters are there in the ...,128 specified characters,56
30185,ASCII,"Originally based on the English alphabet, ASCI...",How many are non-printing control characters?,33 are non-printing control characters,453
30186,ASCII,"Originally based on the English alphabet, ASCI...",How many characters are printable characters?,95 printable characters,561
30187,ASCII,"Originally based on the English alphabet, ASCI...",What is the space also known as what?,invisible graphic:223,630
...,...,...,...,...,...
30288,ASCII,"Unfortunately, requiring two characters to mar...",Why is adding two characters at the ned of a l...,introduces unnecessary complexity and question...,66
30289,ASCII,"Unfortunately, requiring two characters to mar...",What id Multics use to simplify plain text data?,line feed (LF) alone as a line terminator,254
30290,ASCII,"Unfortunately, requiring two characters to mar...",What used carriage return(CR) alone as a line ...,"Macintosh OS, Apple DOS, and ProDOS",395
30291,ASCII,"Unfortunately, requiring two characters to mar...",Who adopted this practice from Multics?,"Unix and Unix-like systems, and Amiga systems",297


In [ ]:
df.to_csv('totaldatas.csv', index=False)

In [ ]:
df_utils.to_csv('utildatas.csv', index=False)